In [1]:
# %pip install xarray
# %pip install matplotlib
# %pip install netcdf4

In [1]:
# -----------------------------------------------------
# 
# Generate lake model input files from CESM outputs
# 
# -----------------------------------------------------

import xarray as xr
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt


### 1. Read in data and extract the variables and coordinate(s) we want

In [2]:
# --- set the lake location (lat / lon, with lon -180 to 180)
latlake, lonlake = 36, -109
n_yrs_repeat = 10    # [yr] number of times to repeat the input data 
                     # (required because the model needs to be spun up)

# +++++++++++++++++++++++++++++++++++++++
# APPLY BIAS CORRECTION?
bias_corr = True
temp_bias = 5.  # [degC] MAT_case - MAT_reanalysis (ERA5 1981-2011)
flds_bias = 27. # [W/m2] FLDS_case - FLDS_reanalysis (ERA5 1981-2011)
# +++++++++++++++++++++++++++++++++++++++

# where to save the .txt file
# [ !! change to match your machine !! ]
savehere = "/Users/tylerkukla/Documents/GitHub/PRYSM/psm/lake_v2/clim_inputs"

# --- read in data
# [ !! change these to be specific to your machine !! ]
datpath = "/Users/tylerkukla/Documents/GitHub/coloplateau-isotopes/cesm_results"   # location of the cesm .nc file
casename = "CP_SLIM_lowTopo_280ppm"                                         # name of the CESM simulation
filename = f"{casename}.cam.h0.0020-0050._01-12_selectVars_climo.nc"

dsin = xr.open_dataset(os.path.join(datpath, filename))

# convert longitude coords to -180 to 180
dsin["lon"] = ((dsin["lon"] + 180) % 360) - 180

# sort the dataset by longitude
dsin = dsin.sortby("lon")
dsin


<xarray.Dataset>
Dimensions:   (lat: 96, lon: 144, lev: 30, month: 12)
Coordinates:
  * lat       (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon       (lon) float64 -180.0 -177.5 -175.0 -172.5 ... 172.5 175.0 177.5
  * lev       (lev) float64 3.643 7.595 14.36 24.61 ... 936.2 957.5 976.3 992.6
  * month     (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables: (12/39)
    UQ        (month, lev, lat, lon) float32 ...
    VQ        (month, lev, lat, lon) float32 ...
    U         (month, lev, lat, lon) float32 ...
    V         (month, lev, lat, lon) float32 ...
    UU        (month, lev, lat, lon) float32 ...
    VV        (month, lev, lat, lon) float32 ...
    ...        ...
    SOLIN     (month, lat, lon) float32 ...
    T         (month, lev, lat, lon) float32 ...
    Z3        (month, lev, lat, lon) float32 ...
    TS        (month, lat, lon) float32 ...
    TREFHT    (month, lat, lon) float32 ...
    PRECT     (month, lat, lon) float32 ...

In [3]:
# --- extract the variables we need as inputs to the lake model
vars_to_keep = ["TREFHT",   # 2m air temperature (2m is the "reference height" or refht)
                "PRECT",    # precipitation
                "RELHUM",   # relative humidity 
                "U10",      # wind speed 
                "FSDS",     # surface incoming shortwave 
                "FLDS",     # surface incoming longwave
                "PS",       # surface pressure
                "QFLX",     # evap (for the runoff calculation)
                ]

dsin = dsin[vars_to_keep]


In [4]:
# --- extract just the coordinate(s) that we want to analyze
dsin = dsin.sel(lat=latlake, lon=lonlake, method='nearest')

# create a new ds to hold the converted values
ds = dsin.copy()

### 2. Convert units to play nice with the lake model

In [5]:
# --- RELHUM: [%]
# pull out just the surface data
ds['RELHUM'] = dsin['RELHUM'].sel(lev=1e3, method='nearest').copy()

# values should already be in percent, but we need to make sure they
# are within ~1 and 100 (trying to avoid values of 0 for the sake of 
# potential divide by zero errors)
ds['RELHUM'] = ds['RELHUM'].where(ds['RELHUM'] >= 1, 1).where(ds['RELHUM'] <= 100, 100).copy()

# see how it looks
# plt.plot(ds.month, ds['RELHUM'])


In [6]:
# --- TREFHT: [degC]
# convert from K to C
# [ ! Dee et al. 2018 supplement notes temperature
#     should be in deg C, but the example input is 
#     in K, so we keep it K for now ! ]
if bias_corr:
    ds['TREFHT'] = dsin['TREFHT'].copy() - temp_bias  # - 273.15
else:
    ds['TREFHT'] = dsin['TREFHT'].copy() # - 273.15

# see how it looks
# plt.plot(ds.month, ds['TREFHT'])

In [7]:
# --- PRECT: [mm]
# convert m/s to mm (assume all months are 30 days for simplicity)
# (conversion factors)
s_per_day = 86400
day_per_month = 30
mm_per_m = 1e3
# convert
ds['PRECT'] = dsin['PRECT'] * mm_per_m * (s_per_day * day_per_month)

# see how it looks
# plt.plot(ds.month, ds['PRECT'])

In [8]:
# --- U10: [m/s]
# U10 var is already in m/s
ds['U10'] = dsin['U10'].copy()

# see how it looks
# plt.plot(ds.month, ds['U10'])

In [9]:
# --- FLDS: [w/m2]
# FLDS var is already in w/m2
if bias_corr:
    ds['FLDS'] = dsin['FLDS'].copy() - flds_bias
else:
    ds['FLDS'] = dsin['FLDS'].copy()

# see how it looks
# plt.plot(ds.month, ds['FLDS'])

In [10]:
# --- FSDS: [w/m2]
# FSDS var is already in w/m2
ds['FSDS'] = dsin['FSDS'].copy()

# see how it looks
# plt.plot(ds.month, ds['FSDS'])

In [11]:
# --- PS: [mb]
# convert Pa to mb
mb_per_Pa = 0.01

# [ ! Dee et al. supplement notes surface 
#     pressure should be in mb but the example
#     input file is in Pa so we'll leave it in 
#     Pa for now ! ]

ds['PS'] = dsin['PS'].copy() # * mb_per_Pa

# see how it looks
# plt.plot(ds.month, ds['PS'])

In [12]:
# --- Runoff: [mm/area of basin]
# convert QFLX kg/m2/s to m/s
density_h2o = 1000     # [kg / m3]

# divide by water density to get m/s
# then convert to mm
ds['QFLX'] = (dsin['QFLX'] / density_h2o) * mm_per_m * (s_per_day * day_per_month)

# get runoff
ds['runoff'] = ds['PRECT'] - ds['QFLX']

# force negative or zero values to something negligible
ds['runoff'] = ds['runoff'].where(ds['runoff'] >= 0, 1e-3).copy()

# see how it looks
# plt.plot(ds.month, ds['runoff'])

### Create a dataframe of the same format as the PRYSM example
For a table of inputs, see: https://agupubs.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1029%2F2018PA003413&file=palo20664-sup-0001-Supplementary.pdf

Dee et al. 2018 supplement states: For the environment sub-model, input rows A through I are required, and presently included. Rows I, J are required to calculate the water balance, and rows K through N are necessary to model stable water isotopes. If water balance and/or isotopes will not be modeled, these rows can either be left blank in the input file or filled with some sort of missing value.

#### Required rows: A-I
Year, Day of year, 2m Temperature, relative humidity, wind speed, surface shortwave, surface longwave, surface pressure, precipitation

#### optional for water balance
precipitation, basin wide runoff

#### optional for water isotopes
d18_p, dD_p, d18_runoff, dD_runoff


In [13]:
# --- build the dataframe

# [ !! NOTE CHECK THE COLUMN POSITION FOR RUNOFF VS PRECIPITATION !! ]

# get rid of the level coord if it exists
if 'lev' in ds.dims:
    ds = ds.drop_dims('lev').copy()
# convert to dataframe
df = ds.to_dataframe().reset_index()

# convert month to day of year
# assume all months have 30 days and we use 
# the 15th day of the month for day of year
df['day_of_year'] = df['month'] * 30 - 15

# add a column for the year
df['year'] = 1

# add water iso columns and leave them blank
df['d18_p'] = -10.
df['dD_p'] = -20.
df['d18_runoff'] = -11.
df['dD_runoff'] = -22.

# order columns
column_order = ['year', 'day_of_year', 'TREFHT', 'RELHUM',
                'U10', 'FSDS', 'FLDS', 'PS', # 'PRECT', 
                'runoff', 'dD_p', 'd18_p', 'PRECT', 
                'dD_runoff', 'd18_runoff']
df = df[column_order]

# round to two decimal places for consistency with Sylvia's inputs
df = df.astype('float').round(2).copy()

# repeat the data n_yrs_repeat times
# (note, using 360 here is chosen to calibrate time steps with the 
#  example in Sylvia's repo.)
df = pd.concat([df.assign(year=df['year'] + i, day_of_year=360 * (i+1) - 360 + df['day_of_year']) for i in range(n_yrs_repeat)], ignore_index=True)

In [14]:
df

,year,day_of_year,TREFHT,RELHUM,U10,FSDS,FLDS,PS,runoff,dD_p,d18_p,PRECT,dD_runoff,d18_runoff
0,1.0,15.0,275.02,51.84,2.02,110.85,253.25,95681.61,21.73,-20.0,-10.0,29.22,-22.0,-11.0
1,1.0,45.0,278.38,47.16,2.23,148.38,265.04,95411.43,29.41,-20.0,-10.0,41.98,-22.0,-11.0
2,1.0,75.0,282.15,37.00,2.50,211.47,270.65,95216.34,8.03,-20.0,-10.0,30.19,-22.0,-11.0
3,1.0,105.0,287.22,30.40,2.60,270.16,288.90,94946.96,0.00,-20.0,-10.0,22.20,-22.0,-11.0
4,1.0,135.0,293.15,28.55,2.55,299.66,319.28,94875.46,0.00,-20.0,-10.0,30.43,-22.0,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,10.0,3465.0,300.85,35.62,2.08,236.45,390.23,95033.20,3.07,-20.0,-10.0,49.15,-22.0,-11.0
116,10.0,3495.0,296.06,34.52,2.21,211.38,350.44,95161.62,10.18,-20.0,-10.0,45.21,-22.0,-11.0
117,10.0,3525.0,288.07,34.42,2.18,178.56,297.39,95370.25,4.84,-20.0,-10.0,25.76,-22.0,-11.0
118,10.0,3555.0,279.11,41.94,1.98,132.99,256.59,95626.54,12.18,-20.0,-10.0,22.57,-22.0,-11.0


In [15]:
# --- save the result
# once with column headers for reference, 
# once without column headers for the 
# lake EBM input file

# make filenames
if bias_corr:
    fn = os.path.join(savehere, f'{casename}_biasCorr_input.txt')
    fn_header = os.path.join(savehere, f'{casename}_biasCorr_input-withHeader.txt')
else:
    fn = os.path.join(savehere, f'{casename}_input.txt')
    fn_header = os.path.join(savehere, f'{casename}_input-withHeader.txt')
# save the header-less version
df.to_csv(fn, header=False, index=False, sep=' ')
# save the header version
df.to_csv(fn_header, header=True, index=False, sep=' ')


In [16]:
# -------------------------------------------